# AB Test Course - PA James 

## 0.1 Imports

In [1]:
import pandas as pd
import math
from statsmodels.stats import api as sms
import numpy as np

# 1.0 Load data

In [10]:
df_raw = pd.read_csv('../datasets/ab_data.csv')

In [11]:
df_raw.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


# 2.0 Design de Experimentos

## 2.1 Formulação da Hipóteses

## 2.2 Parâmetros do Experimento

In [32]:
#Nivel de confiança
confidence_level = 0.95

#Nivel de significancia 
significance_level = 0.05

#Conversoes da pagina atual e da nova pagina(esperado pelo time de negocio)
p1=0.13
p2=0.15

# tamanho do efeito
effect_size = sms.proportion_effectsize(p1,p2)

#Power estatistico (erro de oportunidade)
power = 0.8


In [33]:
#sample size
sample_n = sms.NormalIndPower().solve_power(
    effect_size, 
    power=power,
    alpha=significance_level)

sample_n = math.ceil(sample_n)
sample_n

4720

In [14]:
print('O tamanho total da amostra {}'.format(2*sample_n))
print('O tamanho da amostra do grupo controle é de {}'.format(sample_n))
print('O tamanho da amostra do grupo de tratamento é de {}'.format(sample_n))


O tamanho total da amostra 9440
O tamanho da amostra do grupo controle é de 4720
O tamanho da amostra do grupo de tratamento é de 4720


In [15]:
conversion_rate = 0.10 #quantas vão abrir/responder
x= sample_n / conversion_rate
print('o numero de email para enviar:{}'.format(int(x)))
print('o numero total da base de emial para enviar:{}'.format(2*int(x)))


o numero de email para enviar:47200
o numero total da base de emial para enviar:94400


In [16]:
#Total de pessoas para o estudo com esse parametos são muito alto.
#Podemos aumentar as porcentagem de converções o que não seria ideal, 
#pois uma diferença da p de tratmento deve ser o menor possivel, 
#assim não tendo diferença na aleatoriedade.


# 3.0 Analises descritivas dos dados

In [19]:
print('Number of rows: {}'.format(df_raw.shape[0]))
print('Number of cols: {}'.format(df_raw.shape[1]))

Number of rows: 294478
Number of cols: 5


## 3.1 Verificação de dados faltantes

In [20]:
df_raw.isna().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

## 3.2 Conferir as "flags" do teste A

In [21]:
df_raw[['user_id','group','landing_page']].groupby(['group','landing_page']).count().reset_index()

,group,landing_page,user_id
0,control,new_page,1928
1,control,old_page,145274
2,treatment,new_page,145311
3,treatment,old_page,1965


In [35]:
df_raw_delete = df_raw[['user_id','group']].groupby(['user_id']).count().reset_index().query('group > 1 ')['user_id']
df1 = df_raw[~df_raw['user_id'].isin(df_raw_delete)]
df1.shape

(286690, 5)

In [38]:
df1[['user_id','group','landing_page']].groupby(['group','landing_page']).count().reset_index()

,group,landing_page,user_id
0,control,old_page,143293
1,treatment,new_page,143397


## 3.3 Amonstragem aleatoria dos grupos Controle e Tratamento 

In [48]:
#Contral Group
df_control_sample = df1[df1['group'] == 'control'].sample(n=sample_n, random_state=42)
print('Size of Control Group: {}'.format(df_control_sample.shape[0]))


#Treatment Group
df_treatment_sample = df1[df1['group'] == 'treatment'].sample(n=sample_n, random_state=42)
print('Size of Treatment Group: {}'.format(df_treatment_sample.shape[0]))

#Total Sample Size
df_ab = pd.concat([df_control_sample, df_treatment_sample]).reset_index(drop=True)
df_ab.head()

Size of Control Group: 4720
Size of Treatment Group: 4720


,user_id,timestamp,group,landing_page,converted
0,663037,2017-01-05 02:13:38.063443,control,old_page,0
1,932826,2017-01-13 04:50:06.941679,control,old_page,1
2,922999,2017-01-04 07:19:23.813317,control,old_page,0
3,857826,2017-01-19 02:27:37.678241,control,old_page,0
4,762926,2017-01-18 17:50:59.261406,control,old_page,0


## 3.4 Calculo de métrica de interesse entre os Grupos ( Conversao de cada pagina )

In [54]:
# Control Group
sales = df_control_sample.loc[df_control_sample['converted'] == 1, 'converted'].sum()
visit = len(df_control_sample)

conversion_rate_control = sales/visit
print('Converstion Rate - Control Group: {:.4f}'.format(conversion_rate_control))

#Treatment Group
sales = df_treatment_sample.loc[df_treatment_sample['converted'] == 1, 'converted'].sum()
visit = len(df_treatment_sample)

conversion_rate_treatment = sales/visit
print('Converstion Rate - Control Group: {:.4f}'.format(conversion_rate_treatment))



Converstion Rate - Control Group: 0.1155
Converstion Rate - Control Group: 0.1290
